<h1 style="font-size:30px;">Verifying the ASL Classifier for DPU compatibility</h1>  

In this notebook, we start with a model pre-trained for imagenet, ad verify if it is compatible with the DPU core.  This step is an important sanity check that should be performed BEFORE training.

**The proposed compatibility checking technique is summarized below:**

1. Instantiate our candidate model
2. Inspect model for Vitis-AI_DPU compatibility
3. Add our custom classifier layer for the ASL dataset
4. Inspect model for Vitis-AI_DPU compatibility
5. Iterate until a compatible model has been defined

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/MobileNetV2_03_asl_vitis_ai.png' width=1000 align='center'><br/>

## Table of Contents
* [1 System Configuration](#1-System-Configuration)
* [2 Dataset and Training Configuration](#2-Dataset-and-Training-Configuration)
* [3 Inspecting the Original Model](#4-Inspecing-the-Original-Model)
* [4 Inspecting the Modified Model](#5-Inspecing-the-Modified-Model)
* [5 Conclusion](#6-Conclusion)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import zipfile
import requests
import glob as glob

from tensorflow.keras.utils import image_dataset_from_directory

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)
from dataclasses import dataclass 

block_plot = False
plt.rcParams['figure.figsize'] = (12, 9)
SEED_VALUE = 42 

2023-05-08 08:38:18.973018: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 08:38:19.229766: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## 1 System Configuration

In [2]:
def system_config():
    
    # Get list of GPUs.
    gpu_devices = tf.config.list_physical_devices('GPU')
    print(gpu_devices)
    
    if len(gpu_devices) > 0:
        print('Using GPU')
        os.environ["CUDA_VISIBLE_DEVICES"] = '0'
        os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
        
        # If there are any gpu devices, use first gpu.
        tf.config.experimental.set_visible_devices(gpu_devices[0], 'GPU')
        
        # Grow the memory usage as it is needed by the process.
        tf.config.experimental.set_memory_growth(gpu_devices[0], True)
        
        # Enable using cudNN.
        os.environ['TF_USE_CUDNN'] = "true"
    else:
        print('Using CPU')

system_config()

[]
Using CPU


## 2 Dataset Configuration

In [3]:
@dataclass(frozen=True)
class DatasetConfig:
    NUM_CLASSES: int = 29
    IMG_HEIGHT:  int = 224
    IMG_WIDTH:   int = 224
    CHANNELS:    int = 3
    BATCH_SIZE:  int = 32
    DATA_ROOT:   str = './dataset_ASL_reduced'

## 3 Inspecting the Original Model

### 3.1 Create the original VGG-16 model

The Keras API provides the following utility that is used to instantiate a VGG-16 model, which can be initialized with either pre-trained weights from ImageNet or random initial weights. **In this notebook, we will load the model with weights from the ImageNet dataset** as a starting point for **Training.** For more information on the VGG-16 model available in Keras, refer to the: <a href="https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16" target="_blank">Keras VGG-16 Model API</a>

```python
tf.keras.applications.vgg16.VGG16(include_top=True, 
                                  weights='imagenet', 
                                  input_tensor=None,
                                  input_shape=None, 
                                  pooling=None, 
                                  classes=1000,
                                  classifier_activation='softmax',
                                 )
```

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/MobileNetV2_01_imagenet.png' width=1000 align='center'><br/>

In [4]:
input_shape = (DatasetConfig.IMG_HEIGHT, DatasetConfig.IMG_WIDTH, DatasetConfig.CHANNELS)

print('Create a MobileNetV2 model along with the imagenet pre-trained weights.')
mobilenetv2_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                include_top=True,
                                                alpha=0.5,
                                                weights='imagenet',
                                               )

Create a MobileNetV2 model along with the imagenet pre-trained weights.


2023-05-08 08:38:23.035771: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
print(mobilenetv2_model.summary())

Model: "mobilenetv2_0.50_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 16  432         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 16  64          ['Conv1[0][0]']                  
                                )                                              

 block_3_expand_relu (ReLU)     (None, 56, 56, 96)   0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 96)   0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 96)  864         ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 96)  384         ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 192)  6144        ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 192)  768        ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 192)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 192)  1728       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 192)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 48)   9216        ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 48)  192         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 288)  13824       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 480)   4320        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 480)   1920        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 480)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 80)     38400       ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

### 3.2 Inspect the original VGG-16 model

In [6]:
from tensorflow_model_optimization.quantization.keras import vitis_inspect

In [7]:
!cat ./arch/B512/arch-b512-lr.json

{"fingerprint":"0x101000016010200"}


In [8]:
inspector = vitis_inspect.VitisInspector(target="0x101000016010200") # B512

In [9]:
inspected_model = inspector.inspect_model(mobilenetv2_model)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-083843-328190_WOSYBb.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-083843-328190_WOSYBb.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-083843-328190_WOSYBb.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 973.97it/s]               

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-083843-328190_WOSYBb.xmodel



I20230508 08:38:46.750334 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:38:46.750366 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:38:46.750377 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230508 08:38:46.750685 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model, with op num: 9
I20230508 08:38:46.750710 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:38:46.760483 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230508 08:38:46.760550 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-083843-328190_WOSYBb.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-083843-328190_WOSYBb.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-083843-328190_WOSYBb.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


...

[INFO] generate xmodel     :100%|█| 6/6 [00:00<00:00, 4436.07it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-083843-330647_OdcjGQ.xmodel



I20230508 08:39:05.696430 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:39:05.696460 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:39:05.696470 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230508 08:39:05.696542 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_67, with op num: 6
I20230508 08:39:05.696547 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:39:05.701930 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20230508 08:39:05.701977 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: mobilenetv2_0.50_224
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/155       input_1                 InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/155       Conv1                   Conv2D<linear>          DPU                                                         
------------------------------------------------------------------------------------------------------------------------
2/155       bn_Conv1                BatchNormalization      DPU               

[VAI INFO] Inspected results has been dumped in: inspect_results.txt.
[VAI INFO] Inspect Finished.


In [10]:
!cat ./arch/B128/arch-b128-lr.json

{"fingerprint":"0x101000002010208"}


In [11]:
inspector = vitis_inspect.VitisInspector(target="0x101000002010208") # B128

In [12]:
inspected_model = inspector.inspect_model(mobilenetv2_model)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-083930-284935_QMDeJL.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-083930-284935_QMDeJL.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-083930-284935_QMDeJL.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 1616.43it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-083930-284935_QMDeJL.xmodel



I20230508 08:39:31.585712 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:39:31.585789 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:39:31.585801 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B128_0101000002010208
I20230508 08:39:31.585925 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_68, with op num: 9
I20230508 08:39:31.585935 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:39:31.593547 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230508 08:39:31.593605 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-083930-284935_QMDeJL.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-083930-284935_QMDeJL.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-083930-284935_QMDeJL.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


...

[INFO] generate xmodel     :100%|█| 6/6 [00:00<00:00, 2183.40it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-083930-286954_JfrBdi.xmodel



I20230508 08:39:51.090821 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:39:51.090888 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:39:51.090901 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B128_0101000002010208
I20230508 08:39:51.091060 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_135, with op num: 6
I20230508 08:39:51.091080 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:39:51.096696 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20230508 08:39:51.096769 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: mobilenetv2_0.50_224
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/155       input_1                 InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/155       Conv1                   Conv2D<linear>          DPU                                                         
------------------------------------------------------------------------------------------------------------------------
2/155       bn_Conv1                BatchNormalization      DPU               

[VAI INFO] Inspected results has been dumped in: inspect_results.txt.
[VAI INFO] Inspect Finished.


## 4 Inspecting the Modified Model


### 4.1 VGG Convolutional Base
We begin by creating a model of the VGG-16 convolutional base. We can do this by instantiating the model and setting `include_top = False`, which excludes the fully connected layers. In this notebook, we will instantiate the model with weights that were learned by training the model on the ImageNet dataset.

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/MobileNetV2_10_pretrained_base.png' width=1000 align='center'><br/>

In [13]:
input_shape = (DatasetConfig.IMG_HEIGHT, DatasetConfig.IMG_WIDTH, DatasetConfig.CHANNELS)

print('Create a MobileNetV2 model along with the imagenet pre-trained weights.')
mobilenetv2_model_base = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                    include_top=False,
                                                    alpha=0.5,                                                           
                                                    weights='imagenet',
                                                   )


Create a MobileNetV2 model along with the imagenet pre-trained weights.


In [14]:
print(mobilenetv2_model_base.summary())

Model: "mobilenetv2_0.50_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_156 (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 16  432         ['input_156[0][0]']              
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 16  64          ['Conv1[0][0]']                  
                                )                                              

 block_3_expand_relu (ReLU)     (None, 56, 56, 96)   0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 96)   0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 96)  864         ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 96)  384         ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 192)  6144        ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 192)  768        ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 192)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 192)  1728       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 192)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 48)   9216        ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 48)  192         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 288)  13824       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 480)   4320        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 480)   1920        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 480)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 80)     38400       ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

### 4.2 Add the Classification Layer (attempt 1)
Since we intend to train and use the model to classify hand signals from the ASL dataset (which has 29 classes), we will need to add our own classification layer. In this example, we have chosen to use just a single fully connected dense layer that contains 256 nodes, followed by a softmax output layer that contains 29 nodes for each of the 29 classes. The number of dense layers and the number of nodes per layer is a design choice, but the number of nodes in the output layer must match the number of classes in the dataset.

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/MobileNetV2_02_asl.png' width=1000 align='center'><br/>

In [15]:
print('Add Classification Layers (output or top)')
# Take the output of the MobileNetV2 just before last layer
x = mobilenetv2_model_base.layers[-1].output

#x = tf.keras.layers.Conv2D(filters = 1280,kernel_size = 1,padding='same',use_bias=False, name = 'last_conv')(x)
#x = tf.keras.layers.BatchNormalization(name='last_bn')(x)
#x = tf.keras.layers.ReLU(6,name='last_relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pool')(x)
predictions = tf.keras.layers.Dense(DatasetConfig.NUM_CLASSES,activation='softmax')(x)

model = tf.keras.Model(inputs=mobilenetv2_model_base.input, outputs=predictions)

Add Classification Layers (output or top)


In [16]:
print(model.summary())

Model: "model_136"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_156 (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 16  432         ['input_156[0][0]']              
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 16  64          ['Conv1[0][0]']                  
                                )                                                         

 block_3_expand_relu (ReLU)     (None, 56, 56, 96)   0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 96)   0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 96)  864         ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 96)  384         ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 192)  6144        ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 192)  768        ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 192)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 192)  1728       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 192)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 48)   9216        ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 48)  192         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 288)  13824       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 480)   4320        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 480)   1920        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 480)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 80)     38400       ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

### 4.3 Inspect the Modified Model (attempt 1)

In [17]:
inspector = vitis_inspect.VitisInspector(target="0x101000016010200") # B512

In [18]:
inspected_model = inspector.inspect_model(model)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084019-019618_UEnuVy.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084019-019618_UEnuVy.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084019-019618_UEnuVy.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 1656.52it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084019-019618_UEnuVy.xmodel



I20230508 08:40:20.454715 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:40:20.454761 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:40:20.454771 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230508 08:40:20.454850 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_137, with op num: 9
I20230508 08:40:20.454859 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:40:20.463955 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230508 08:40:20.464002 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084019-019618_UEnuVy.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084019-019618_UEnuVy.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084019-019618_UEnuVy.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


...

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084019-022541_sxFNBD.xmodel


I20230508 08:40:37.564754 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:40:37.564778 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:40:37.564786 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230508 08:40:37.564857 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_203, with op num: 6
I20230508 08:40:37.564862 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:40:37.567564 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20230508 08:40:37.567597 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084019-022541_sxFNBD.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084019-022541_sxFNBD.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084019-022541_sxFNBD.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 6/6 [00:00<00:00, 2629.38it/s]              
I20230508 08:40:37.764840 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:40:37.764873 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:40:37.764880 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230508 08:40:37.765051 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_204, with op num: 6
I20230508 08:40:37.765059 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:40:37.768198 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20230508 08:40:37.768239 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084019-022541_sxFNBD.xmodel
[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: model_136
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/155       input_156               InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/155       Conv1                   Conv2D<linear>          DPU                                                         
---------------------------------------------------------------------------------------

In [19]:
inspector = vitis_inspect.VitisInspector(target="0x101000002010208") # B128

In [20]:
inspected_model = inspector.inspect_model(model)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084049-531329_wSKvGR.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084049-531329_wSKvGR.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084049-531329_wSKvGR.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 2692.11it/s]              
I20230508 08:40:50.129591 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:40:50.129618 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:40:50.129629 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B128_0101000002010208
I20230508 08:40:50.129770 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_205, with op num: 9
I20230508 08:40:50.129781 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:40:50.139168 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230508 08:40:50.139209 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084049-531329_wSKvGR.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084049-531329_wSKvGR.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084049-531329_wSKvGR.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084049-531329_wSKvGR.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 1235.36it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084049-531329_wSKvGR.xmodel


...

[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084049-532416_JhrNZp.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084049-532416_JhrNZp.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084049-532416_JhrNZp.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 6/6 [00:00<00:00, 3479.79it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084049-532416_JhrNZp.xmodel



I20230508 08:40:59.629856 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:40:59.629878 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:40:59.629884 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B128_0101000002010208
I20230508 08:40:59.629947 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_272, with op num: 6
I20230508 08:40:59.629952 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:40:59.632789 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20230508 08:40:59.632814 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: model_136
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/155       input_156               InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/155       Conv1                   Conv2D<linear>          DPU                                                         
------------------------------------------------------------------------------------------------------------------------
2/155       bn_Conv1                BatchNormalization      DPU                          

### 4.4 Add the Classification Layer (attempt 2)
Since we intend to train and use the model to classify hand signals from the ASL dataset (which has 29 classes), we will need to add our own classification layer. In this example, we have chosen to use just a single fully connected dense layer that contains 256 nodes, followed by a softmax output layer that contains 29 nodes for each of the 29 classes. The number of dense layers and the number of nodes per layer is a design choice, but the number of nodes in the output layer must match the number of classes in the dataset.

It turns out that during model compilation, the dense layer of size 1280 is not supported on the smaller DPU B128 architecture.

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/MobileNetV2_02_asl_B128_issue.png' width=1000 align='center'><br/>

For this reason we reduce the size of the last layers from 1280 to 1000.

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/MobileNetV2_03_asl_vitis_ai.png' width=1000 align='center'><br/>

In [21]:
print('Add Classification Layers (output or top)')
# Take the output of the MobileNetV2 before last layers
x = mobilenetv2_model_base.layers[-4].output

x = tf.keras.layers.Conv2D(filters = 1000,kernel_size = 1,padding='same',use_bias=False, name = 'last_conv')(x)
x = tf.keras.layers.BatchNormalization(name='last_bn')(x)
x = tf.keras.layers.ReLU(6,name='last_relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pool')(x)
predictions = tf.keras.layers.Dense(DatasetConfig.NUM_CLASSES,activation='softmax')(x)

model = tf.keras.Model(inputs=mobilenetv2_model_base.input, outputs=predictions)

Add Classification Layers (output or top)


In [22]:
print(model.summary())

Model: "model_273"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_156 (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 16  432         ['input_156[0][0]']              
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 16  64          ['Conv1[0][0]']                  
                                )                                                         

 block_3_expand_relu (ReLU)     (None, 56, 56, 96)   0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 96)   0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 96)  864         ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 96)  384         ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 192)  6144        ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 192)  768        ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 192)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 192)  1728       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 192)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 48)   9216        ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 48)  192         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 288)  13824       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 480)   4320        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 480)   1920        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 480)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 80)     38400       ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

### 4.5 Inspect the Modified Model (attempt 2)

In [23]:
inspector = vitis_inspect.VitisInspector(target="0x101000016010200") # B512

In [24]:
inspected_model = inspector.inspect_model(model)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084110-573202_RgqyEp.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084110-573202_RgqyEp.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084110-573202_RgqyEp.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 1406.54it/s]              
I20230508 08:41:11.171788 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:41:11.171826 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:41:11.171840 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230508 08:41:11.171945 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_274, with op num: 9
I20230508 08:41:11.171958 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:41:11.180979 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230508 08:41:11.181026 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084110-573202_RgqyEp.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084110-573202_RgqyEp.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084110-573202_RgqyEp.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084110-573202_RgqyEp.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 2206.83it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084110-573202_RgqyEp.xmodel


...

[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084110-574361_RXthrq.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084110-574361_RXthrq.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084110-574361_RXthrq.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 6/6 [00:00<00:00, 2976.09it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084110-574361_RXthrq.xmodel



I20230508 08:41:20.516675 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:41:20.516705 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:41:20.516716 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230508 08:41:20.516798 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_341, with op num: 6
I20230508 08:41:20.516808 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:41:20.521246 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20230508 08:41:20.521279 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: model_273
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/155       input_156               InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/155       Conv1                   Conv2D<linear>          DPU                                                         
------------------------------------------------------------------------------------------------------------------------
2/155       bn_Conv1                BatchNormalization      DPU                          

In [25]:
inspector = vitis_inspect.VitisInspector(target="0x101000002010208") # B128

In [26]:
inspected_model = inspector.inspect_model(model)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084131-353280_zaLtGT.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084131-353280_zaLtGT.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084131-353280_zaLtGT.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 2234.34it/s]              
I20230508 08:41:31.957051 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:41:31.957084 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:41:31.957098 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B128_0101000002010208
I20230508 08:41:31.957193 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_342, with op num: 9
I20230508 08:41:31.957206 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:41:31.966328 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230508 08:41:31.966359 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084131-353280_zaLtGT.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084131-353280_zaLtGT.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084131-353280_zaLtGT.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084131-353280_zaLtGT.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 2267.19it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084131-353280_zaLtGT.xmodel


...

[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084131-356584_Aydqjs.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084131-356584_Aydqjs.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230508-084131-356584_Aydqjs.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 6/6 [00:00<00:00, 3828.08it/s]              

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230508-084131-356584_Aydqjs.xmodel



I20230508 08:41:41.736312 51671 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230508 08:41:41.736343 51671 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230508 08:41:41.736353 51671 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B128_0101000002010208
I20230508 08:41:41.736433 51671 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_409, with op num: 6
I20230508 08:41:41.736443 51671 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230508 08:41:41.740839 51671 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20230508 08:41:41.740870 51671 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: model_273
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/155       input_156               InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/155       Conv1                   Conv2D<linear>          DPU                                                         
------------------------------------------------------------------------------------------------------------------------
2/155       bn_Conv1                BatchNormalization      DPU                          

## 5 Conclusion

In this notebook, we showed how you can use Vitis-AI's **model-inspector** to ensure compatibility with the DPU architecture.  Performing this step PRIOR to training saves a lot of pain and time. 